In [1]:
import tensorflow as tf
import keras
from keras import models
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model, Model
from keras.layers import *
import os
import sys
import numpy as np
from score_dataset_helpers import *
import pandas as pd

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
base_dir = 'C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/data_dual_inputs/'
train_dir = base_dir + 'train'
validation_dir = base_dir + 'validation'
test_dir = base_dir + 'test'

## Define model

In [3]:
f1, f2, f3 = 8,16,32
k_size = (3,3)

pics_input = Input(shape = (75, 250, 3))
pics_c1 = Conv2D(filters = f1, kernel_size = k_size, activation = "relu")(pics_input)
pics_m1 = MaxPooling2D(k_size)(pics_c1)
pics_c2 = Conv2D(filters = f2, kernel_size = k_size, activation = "relu")(pics_m1)
pics_m2 = MaxPooling2D(k_size)(pics_c2)
#pics_c3 = Conv2D(filters = f3, kernel_size = k_size, activation = "relu")(pics_m2)
#pics_m3 = MaxPooling2D((2,2))(pics_c3)
pics_f = Flatten()(pics_m2)
pics_d = Dense(20, activation = "relu")(pics_f)

nums_input = Input(shape = ([1]))
nums_features = Dense(5, activation = "relu")(nums_input)

conc_layer = concatenate([pics_d, nums_features])
dense_1 = Dense(100, activation = "relu")(conc_layer)
#dense_12 = Dense(100, activation = "softmax")(dense_1)
dense_2 = Dense(1, activation = "sigmoid")(dense_1)

model = Model(inputs = [pics_input, nums_input], outputs = dense_2)
model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["acc"])

In [4]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 75, 250, 3)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 73, 248, 8)   224         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 24, 82, 8)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 22, 80, 16)   1168        max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

## Create vectors of scores from pictures

In [5]:
train_scores = train_dir + "/score"
train_pics = train_dir + "/pics"

In [6]:
train_names_scores = [f for f in os.listdir(train_scores)]
train_names_pics = [f for f in os.listdir(train_pics)]

In [7]:
print(str(len(train_names_scores)) + " training files for scores")
print(str(len(train_names_pics)) + " training files for pictures")

517 training files for scores
517 training files for pictures


In [28]:
train_dataset_score = create_dataset("score", train_names_scores, base_dir + "train/")

In [29]:
def get_result(result):
    resultstr = ''
    for i in range(5):
        resultstr += str(np.argmax(result[i]))
    return resultstr

In [30]:
score_model = load_model("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/models/score_v2.h5")

In [31]:
#Create vectors for scores and for labels
score_vec = []
for i in range(len(train_names_scores)):
    y_pred = score_model.predict(train_dataset_score[i].reshape(1, 50, 160, 1))
    score_vec.append(get_result(y_pred))

label_vec = ['up' in x for x in train_names_pics]
label_vec = np.array(label_vec)
label_array = label_vec.astype(float)

In [32]:
print_dat = pd.DataFrame({'score' : score_vec, 'label' : label_array, 'orig_name' : train_names_scores})
print_dat

,label,orig_name,score
0,0.0,right_1.png,00008
1,0.0,right_10.png,00073
2,0.0,right_100.png,00172
3,0.0,right_101.png,00185
4,0.0,right_102.png,00190
5,0.0,right_105.png,00200
6,0.0,right_106.png,00200
7,0.0,right_107.png,00226
8,0.0,right_108.png,00238
9,0.0,right_109.png,00243


In [33]:
pic_array = np.zeros((len(train_names_pics), 75, 250, 3))

for i in range(len(train_names_pics)):
    x = load_img(train_pics + '/' + train_names_pics[i], target_size = (75, 250))
    #x = img_to_array(x)
    pic_array[i] = x

In [34]:
pic_array[1][50]

array([[255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [100., 100., 100.],
       [ 83.,  83.,  83.],
       [ 83.,  83.,  83.],
       [ 83.,  83.,  83.],
       [ 83.,  83.,  83.],
       [ 83.,  83.,  83.],
       [ 83.,  83.,  83.],
       [ 83.,  83.,  83.],
       [100., 100., 100.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
       [255., 255., 255.],
 

In [35]:
pic_array = np.array(pic_array)
pic_array /= 255
score_array = np.array(score_vec)
score_array = np.float32(score_array)

In [36]:
pic_array.shape

(517, 75, 250, 3)

In [37]:
score_array.shape

(517,)

In [38]:
label_array.shape

(517,)

In [45]:
model.fit([pic_array, score_array], label_array, epochs = 15, shuffle = True, batch_size = 100, validation_split = 0.1)

Train on 465 samples, validate on 52 samples
Epoch 1/15
465/465 [==============================] - 4s 8ms/step - loss: 0.4416 - acc: 0.8237 - val_loss: 1.0897 - val_acc: 0.0769
Epoch 2/15
465/465 [==============================] - 4s 8ms/step - loss: 0.3906 - acc: 0.8344 - val_loss: 0.8048 - val_acc: 0.5577
Epoch 3/15
465/465 [==============================] - 4s 8ms/step - loss: 0.3286 - acc: 0.8710 - val_loss: 0.5194 - val_acc: 0.7692
Epoch 4/15
465/465 [==============================] - 4s 8ms/step - loss: 0.2558 - acc: 0.9247 - val_loss: 0.3519 - val_acc: 0.8654
Epoch 5/15
465/465 [==============================] - 4s 8ms/step - loss: 0.1980 - acc: 0.9398 - val_loss: 0.4067 - val_acc: 0.8077
Epoch 6/15
465/465 [==============================] - 4s 8ms/step - loss: 0.1652 - acc: 0.9462 - val_loss: 0.5860 - val_acc: 0.7885
Epoch 7/15
465/465 [==============================] - 4s 8ms/step - loss: 0.1469 - acc: 0.9505 - val_loss: 0.5282 - val_acc: 0.8077
Epoch 8/15
465/465 [===========

In [53]:
model.save("C:/Users/bened/OneDrive/Arbeit/Lernen/python_training/models/model_dual_input.h5")

In [57]:
model.predict([pic_array[1], score_array[1]])

ValueError: Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (75, 250, 3)